# Visualization with Streamlit

In [1]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.4 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 14.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
%%writefile 'src/streamlit-app.py'

import streamlit as st
import digitalhub as dh
import getpass as gt
import pandas as pd

PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

prediction_nbeats = proj.get_dataitem("parking_prediction_nbeats_model")
prediction_sarimax_regression = proj.get_dataitem("parking_data_predicted_regression")

rdf1 = prediction_sarimax_regression.as_df();

rdf2 = prediction_nbeats.as_df()
rdf2.datetime = pd.to_datetime(rdf2.datetime, unit='ms')

st.write("""prediction Sarimax regression""")
st.line_chart(rdf1, x="datetime", y="predicted_mean")

st.write("""prediction Nbeats model""")
st.line_chart(rdf2, x="datetime", y="predicted_mean")


Writing src/streamlit-app.py


In [2]:
!streamlit run src/streamlit-app.py --browser.gatherUsageStats false


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.3.132:8501
  External URL: http://172.213.252.90:8501

created project parcheggi-nk-scheduler-tenant2
^C
  Stopping...


In [2]:
import digitalhub as dh
import getpass as gt

PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

created project parcheggi-nk-scheduler-khurshid


In [3]:
func = proj.new_function(name="streamlit-viz-v3",
                         kind="container",
                         # baseImage="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest"
                         image="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest",
                         code_src="src/streamlit-app.py", args= ["run", "dashboard-viz.py"], command= "streamlit")

In [4]:
func.spec.to_dict()

{'image': 'ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest',
 'command': 'streamlit',
 'args': ['run', 'dashboard-viz.py'],
 'source': {'source': 'src/streamlit-app.py',
  'base64': 'CmltcG9ydCBzdHJlYW1saXQgYXMgc3QKaW1wb3J0IGRpZ2l0YWxodWIgYXMgZGgKaW1wb3J0IGdldHBhc3MgYXMgZ3QKaW1wb3J0IHBhbmRhcyBhcyBwZAoKUFJPSkVDVF9OQU1FID0gInBhcmNoZWdnaS1uay1zY2hlZHVsZXItIitndC5nZXR1c2VyKCkKcHJvaiA9IGRoLmdldF9vcl9jcmVhdGVfcHJvamVjdChQUk9KRUNUX05BTUUpCnByaW50KCJjcmVhdGVkIHByb2plY3Qge30iLmZvcm1hdChQUk9KRUNUX05BTUUpKQoKcHJlZGljdGlvbl9uYmVhdHMgPSBwcm9qLmdldF9kYXRhaXRlbSgicGFya2luZ19wcmVkaWN0aW9uX25iZWF0c19tb2RlbCIpCnByZWRpY3Rpb25fc2FyaW1heF9yZWdyZXNzaW9uID0gcHJvai5nZXRfZGF0YWl0ZW0oInBhcmtpbmdfZGF0YV9wcmVkaWN0ZWRfcmVncmVzc2lvbiIpCgpyZGYxID0gcHJlZGljdGlvbl9zYXJpbWF4X3JlZ3Jlc3Npb24uYXNfZGYoKTsKCnJkZjIgPSBwcmVkaWN0aW9uX25iZWF0cy5hc19kZigpCnJkZjIuZGF0ZXRpbWUgPSBwZC50b19kYXRldGltZShyZGYyLmRhdGV0aW1lLCB1bml0PSdtcycpCgpzdC53cml0ZSgiIiJwcmVkaWN0aW9uIFNhcmltYXggcmVncmVzc2lvbiIiIikKc3QubGluZV9jaG

In [ ]:
#fun.run(action="serve", command='streamlit run src/streamlit-app.py --browser.gatherUsageStats false', )

In [6]:
func.run(action="serve", service_port=[{"port": 5002, "target_port": 5801}])
#service_type = "NodePort"

## Expose the service

To make it accessible from outside, we'll need to create an API gateway.

Go to the Kubernetes Resource Manager component (available from dashboard) and go to the API Gateways section. To expose a service it is necessary to define

name of the gateway
the service to expose
the endpoint where to publish
and the authentication method (right now only no authentication or basic authentication are available). in case of basic authentication it is necessary to specify Username and Password.
The platform by default support exposing the methods at the subdomains of services.<platform-domain>, where platform-domain is the domain of the platform instance.

<img src="streamlit.jpg">

Save and, after a few moments, you will be able to call the API at the address you defined! If you set Authentication to Basic, don't forget that you have to provide the credentials.